In [ ]:
import urllib, http.client
import time
import json
# эти модули нужны для генерации подписи API
import hmac, hashlib
import sys
import copy
import pandas as pd
import csv
sys.path

In [ ]:
API_KEY ='K-3038939b9a19156293f232b73e6d2392c9799078'
API_SECRET = 'S-c25c5229f8be56020bcd495c8adac52e02ff0c4a'

In [ ]:
CURRENCY_1 = 'BTC'
CURRENCY_2 = 'RUB'
CURRENCY_1_MIN_QUANTITY = 0.001

In [ ]:
ORDER_LIFE_TIME = 3 # минуты
STOCK_FEE = 0.002 # комиссия == 0.2%
AVG_PRICE_PERIOD = 90 # за какой период брать ср цену(минуты)
CAN_SPEND = 5 # сколько тратить CUR_2 при каждой покупке CUR_1
BLOCK_SPEND = 30  # ограничение CUR_2 на покупку
PROFIT_MARKUP = 0.001 # прибыль в 1%
DEBUG = True # инфа отладки

STOCK_TIME_OFFSET = 0 # если время биржи расходится с нашим
# обращение к api
API_URL = 'api.exmo.me'
API_VERSION = 'v1'

CURRENT_PAIR = CURRENCY_1 + '_' + CURRENCY_2

In [ ]:
class ExmoAPI:
#     тут всё понятно
    def __init__(self, API_KEY, API_SECRET, API_URL = 'api.exmo.me', API_VERSION = 'v1'):
        self.API_URL = API_URL
        self.API_VERSION = API_VERSION
        self.API_KEY = API_KEY
        self.API_SECRET = bytes(API_SECRET, encoding='utf-8')
        
# Из строки payload получаем "подпись", хешируем с помощью секретного ключа API
# sing - получаемый ключ, который будет отправлен на биржу для проверки
    def sha512(self, data):
        H = hmac.new(key = self.API_SECRET, digestmod = hashlib.sha512)
        H.update(data.encode('utf-8'))
        return H.hexdigest()
    
# Составляем словарь {ключ:значение} для отправки на биржу

    def api_query(self, api_method, params = {}):
        params['nonce'] = int(round(time.time() * 1000))
        # Переводим словарь payload в строку, в формат для отправки через GET/POST и т.п.
        params =  urllib.parse.urlencode(params)

        sign = self.sha512(params)
        
        # Формируем заголовки request для отправки запроса на биржу. 
        # Передается публичный ключ API и подпись, полученная с помощью hmac
        headers = {
            "Content-type": "application/x-www-form-urlencoded",
            "Key": self.API_KEY,
            "Sign": sign
        }
# Создаем подключение к бирже, если в течении 60 сек не удалось подключиться, обрыв соединения    
        conn = http.client.HTTPSConnection(self.API_URL, timeout=60)
    # После установления связи, запрашиваем переданный адрес
    # В заголовке запроса уходят headers, в теле - payload
        conn.request("POST", "/" + self.API_VERSION + "/" + api_method, params, headers)
         # Получаем ответ с биржи и читаем его в переменную response
        response = conn.getresponse().read()
        conn.close()
        
        try:
            obj = json.loads(response.decode('utf-8'))
            # Смотрим, есть ли в полученном объекте ключ "error"
            if 'error' in obj and obj['error']:
                print(obj['error'])
                raise sys.exit()
            return obj
        # Если не удалось перевести полученный ответ (вернулся не JSON)
        except json.decoder.JSONDecodeError:
            print('Error while parsing response:', response)
            raise sys.exit()

In [ ]:
api = ExmoAPI(API_KEY, API_SECRET)
# print(ExmoAPI_instance.api_query('user_info'))
print(api.api_query('user_info')['server_date'])


# трейды

In [ ]:
def create_trades_interval(order):
    interval = {}
    interval['trades'] = []
    return interval

In [ ]:
def upgrade_trades_interval(order, interval):
    interval['trades'].append(order)
    return interval

In [ ]:
interval_timer =  1000 
limit = 700 # ограничение подгружаемых ордеров

# ордера

In [ ]:
def orders_to_interval(interval):
    bid = []
    ask = []
    
    for string in api.api_query('order_book',{'pair':str(CURRENCY_1)+'_'+str(CURRENCY_2),'limit':limit}).items():
        for order in string[1].items():
            if order[0] == 'ask':
                for i in order[1]:                
                    ask.append(i)
            elif order[0] == 'bid':
                for i in order[1]:
                    bid.append(i)
    
    interval['ask_orders'] = ask
    interval['bid_orders'] = bid
    
    return interval
    

# запись

In [ ]:
trade_time = {}

while True:
    time.sleep(interval_timer) # ожидание нового интервала
    for string in api.api_query('trades',{'pair':str(CURRENCY_1)+'_'+str(CURRENCY_2),'limit':500}).items():
        interval_open = api.api_query('user_info')['server_date'] # Время на сервере сейчас
        interval_close = interval_open - interval_timer # Время закрытия интервала
        for order in string[1]:
            # Если входит в интервал
            if order['date'] >= interval_close and order['date'] < interval_open:                
                if trade_time == {}:
                    #Создает поля
                    trade_time['%s' % (interval_open)] = create_one_interval(order)
                else:
                    # Обновляет поля
                    trade_time['%s' % (interval_open)] = upgrade_one_interval(order,(trade_time['%s' % (interval_open)]))
                    
#     подключаем ордера
    orders_to_interval(trade_time['%s' % (interval_open)])
        
#     запись в файл
    df = pd.DataFrame.from_dict(trade_time)
    df = df.T
    df.to_csv('result.csv',mode='a')
    trade_time = {} 
    